## Исследование датасета со стартапами для прогнозирвоания закрытия и составления портрета успешного стартапа

Проблемы в данных:

- несбалансированные выборки: открытых стратапов 90%, закрытых 10%
- пропуски составляли 10%, поле с объемом инвестиций - 20%
- выбросы 2,6%
- столбец категория содержал 4191 уникальную категорию
- столбцы с датами также содержали множество уникальных значений, что усложняло дерево решений

Чтобы решить задачу, я решила сделать MVP, с минимальными изменениями/предобработкой данных для оценки того, какие способы и на сколько улучшают предсказательную способность модели, используем подход Continuous Improvement

## Первая итерация:

- привела столбцы с датами к формату дат
- необходимо было сгенерировать новый столбец lifetime  в тренировочном датасете, так как он присутствовал в тестовом, для этого пропущенные значения в поле closed_at заполнила датой выгрузки, а столбец lifetime = 'closed_at' - 'founded_at'

**Вывод по обучению первой модели**
- Обучили методом RandomForestClassifier, DecisionTreeClassifier, KNeighborsClassifier
- параметры лучшей модели 'max_depth': 11, 'min_samples_leaf': 3, 'min_samples_split': 8
- метрика лучшей модели на кроссвалидационной выборке - Best score: 80%. При проверке на тестовой выборки метрика составила 71,33 
Минимальный порог был преодолен, но решила улучшить показатель, попробовав новые модели

По результатам поиска лучшей модели в первом этапе определила что модель RandomForestClassifier показывает себя лучше всего. исходя из этих данных было сделано предположение, что лучшей моделью нечувствительной к пропускам и большому кол-ву категорий в данных, являются модели основанные на градиентном бустинге.

Справка про градиентный бустинг:

В ходе обучения случайного леса каждый базовый алгоритм строится независимо от остальных. Бустинг, в свою очередь, воплощает идею последовательного построения линейной комбинации алгоритмов. Каждый следующий алгоритм старается уменьшить ошибку текущего ансамбля.

Бустинг, использующий деревья решений в качестве базовых алгоритмов, называется градиентным бустингом над решающими деревьями, Gradient Boosting on Decision Trees, GBDT. Он отлично работает на выборках с «табличными», неоднородными данными. Примером таких данных может служить описание пользователя Яндекса через его возраст, пол, среднее число поисковых запросов в день, число заказов такси и так далее. Такой бустинг способен эффективно находить нелинейные зависимости в данных различной природы. Этим свойством обладают все алгоритмы, использующие деревья решений, однако именно GBDT обычно выигрывает в подавляющем большинстве задач. Благодаря этому он широко применяется во многих конкурсах по машинному обучению и задачах из индустрии (поисковом ранжировании, рекомендательных системах, таргетировании рекламы, предсказании погоды, пункта назначения такси и многих других).

## Вторая итерация CatBoostClassifier + Optuna

Поиск гиперпараметров осуществлялся через Optuna: получилось быстрее чем GridSearchCV примерно в 3 раза. Удалось улучшить метрику F1 на тестовой выборке на 5% - до 76,81%

## Третья итерация XGBClassifier + Optuna

Поиск гиперпараметров осуществлялся через Optuna. Метрика F1 на тестовой выборке - 76,82%

Применение другого метода поиска гиперпараметров и других моделей увеличило предсказательную способность модели незначительно.

Модели считались очень долго, что также ухудшало возможность проверки гипотез. Необходимо было решать проблему с большим количеством категорий и искать новые признаки:
- добавлен новый синтетический признак, который позволил использовать столбцы с датами инвестиционных раундов: 
    - delta_funding_time - время между раундами, показывает косвенно на то, что стартап долгое время пользуется популярностью и оправдывает ожидания инвесторов своей работой
    - delta_invest&open_time' - кол-во дней между датой открытия и первым инвестиционным раундом - может косвенно указывать на то, что стартап был замечен инвесторами и находится в востребованной категории бизнеса
    - delta_invest_last_open_time - в дальнейшем был исключен так как ухудшал метрику на 2%
    - founded_year - год основания компании, брала именно год так как обычно влияние на стартапы оказывает ситуация на финансовом рынке в тот или иной финансовый год, если есть кризис, то инвесторы рискуют реже, если ситуация стабильна и прогнозируема то инвесторы чаще вкладываются, а инвестиции дают возможность стартапам развиваться и захватывать новые потребительские рынки
    - new_category была добавлена чтобы уменьшить разреженность матрицы после энкодинга. В столбце присутствовали составные категории, которые включали в себя несколько значений разделенных |, я решила реализовать решение, которое бы в этом поле оставляло самое распространненное значение, если оно состоит из нескольких категорий + в шаге с описанием кодирования задала ппараметр, ограничивающий создание новых категорий, у которых частота меньше 500
- построила матрицу корреляций, на которой были выявлены признаки с мультиколлинеарностью, что в некоторых случая логично, так как новые синтетические признаки были получены из них. На основе матрицы было решено удалить столбцы:
    - 'category_list'
    - 'state_code'
    - 'region'
    - 'city'
    - 'founded_at'
    - 'first_funding_at'
    - 'last_funding_at'
    - 'closed_at'
- обучила модель CatBoostClassifier, метрика на тестовой выборке - 96,47
- обучила модель XGBClassifier, метрика на тестовой выборке - 99,099

Следующие шаги не дали прироста в прогнозной способности модели, а где-то ухудшили метрику
- удалить признаки, которые имеют маленький вес - оставила 4 признака: lifetime, founded_year, delta_invest&open_time, delta_funding_time
- оставила 2 признака:  lifetime, founded_year,
- оставила 35 признаков отобранных с помощью SelectKBest


Что можно было еще  реализовать:
- когда строила таблицу важности признаков, то был выявлен признак с отсутствующей категорией, и ему модель присваивала большой вес. Как улучшение можно было попробовать не решать топорно указанием в OHE частоты уникального значения, а все же семанически агрегировать категории в более крупные. Также была идея использовать get_dummies, но есть ограничения этого метода - на новых категориях, на которых он не обучался - выдаст ошибку, но можно поискать обходы или новые библиотеки
- не удалось применить параметр в CatBoostClassifier - cat_features
- собрать статистические данные проанализировав shap вывод для определения на каких порогах стартап успешнее по ключевым признакам, возможно выделить коридорные значения, например, если delta_funding_time > 200, то стартап будет успешным и тд
- 
